In [ ]:
import os
import sys
import pandas as pd
import xarray as xr
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.getcwd(), ".."))
from models.components.features_selection import lcsi,discretize_data,JMI_score,model_selection_L2reg,hjmi_selection,backward_model_selection,elbow_plot
from shared_utils.utils_data import extract_index_label, feature_checker
from sklearn.feature_selection import mutual_info_regression
import infoselect as inf

In [ ]:
metrics = "/workspaces/ecg_evaluation/results/quality_metrics.nc"
d = xr.load_dataset(metrics)
list_features = [
        "Corr_interlead",
        "Corr_intralead",
        "wPMF",
        "SNRECG",
        "HR",
        "Flatline",
        "TSD",
    ]
df_X_mean, df_y = extract_index_label(
        d, list_features, aggregation_method="mean"
    )
initial_feature_set = list(df_X_mean.columns.values)
print(df_X_mean.columns.values)

In [ ]:
corr = df_X_mean.corr()
sns.heatmap(corr,xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,annot=True)

In [ ]:
df_X_mean["der_label"] = df_y.values

In [ ]:
initial_feature_set = list(df_X_mean.columns.values)
x_dis = discretize_data(df_X_mean)
plt.figure()
fig1,ax = plt.subplots(nrows = 2,ncols = 4,figsize=(15,15))
coordinates = [(x,y) for y in range(4) for x in range(2)]
color = iter(plt.cm.rainbow(np.linspace(0, 1, 7)))
for j in range(x_dis.shape[1]):
    ax[coordinates[j]].hist(x_dis.iloc[:,j])
    ax[coordinates[j]].set_title(initial_feature_set[j])

plt.show()

In [ ]:
initial_feature_set = list(df_X_mean.columns.values)
print(df_X_mean)
x_dis = discretize_data(df_X_mean)
F, JMI, MI = lcsi(x_dis.values, df_y.values.ravel(),function_name =  'JMI')
print(np.array(initial_feature_set)[F])
print(JMI)
print(MI)

In [ ]:
rep = backward_model_selection(df_X_mean,df_y)
print(rep)

In [ ]:
test = "/workspaces/ecg_evaluation/results/evaluation_metrics"
rep = JMI_score(df_X_mean,df_y)
print(rep)

In [ ]:
test = "/workspaces/ecg_evaluation/results/evaluation_metrics"
rep = hjmi_selection(df_X_mean,df_y)
print ([initial_feature_set[i] for i in rep[0]])
elbow_plot(rep[2])

In [ ]:
rep = model_selection_L2reg(df_X_mean,df_y)
print(rep)